<a href="https://colab.research.google.com/github/novrmd/GitHubUser/blob/master/sic57_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Dependecies

In [ ]:
import pandas as pd
import zipfile
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nooviidn","key":"b2382e92c6d1ee3d4d43ae4b333edcfc"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [ ]:
!kaggle datasets download -d abhisheksjha/time-series-air-quality-data-of-india-2010-2023

Dataset URL: https://www.kaggle.com/datasets/abhisheksjha/time-series-air-quality-data-of-india-2010-2023
License(s): CC-BY-NC-SA-4.0
 99% 461M/464M [00:05<00:00, 110MB/s]
100% 464M/464M [00:05<00:00, 81.9MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('time-series-air-quality-data-of-india-2010-2023.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

Data Collection and Processing

In [ ]:
# Load the dataset
def combine_state_data(dataset_src):
    cg_files = glob.glob(os.path.join(dataset_src, 'CG*.csv'))
    print(f'Combining a total of {len(cg_files)} files...\n')

    combined_df = []

    for cg_file in cg_files:
        print(f"File: {cg_file}")
        file_df = pd.read_csv(cg_file)
        combined_df.append(file_df)

    df_combined = pd.concat(combined_df, ignore_index=True)

    return df_combined

In [ ]:
dataset_src = '/content'
df = combine_state_data(dataset_src)

Combining a total of 14 files...

File: /content/CG012.csv
File: /content/CG005.csv
File: /content/CG003.csv
File: /content/CG011.csv
File: /content/CG010.csv
File: /content/CG014.csv
File: /content/CG004.csv
File: /content/CG008.csv
File: /content/CG001.csv
File: /content/CG006.csv
File: /content/CG009.csv
File: /content/CG007.csv
File: /content/CG013.csv
File: /content/CG002.csv


In [ ]:
#print 5 first row of the dataset
df.head()

,From Date,To Date,PM2.5 (ug/m3),PM10 (ug/m3),NO (ug/m3),NO2 (ug/m3),NOx (ppb),NH3 (ug/m3),SO2 (ug/m3),CO (mg/m3),...,RF (mm),CO (mg/Nm3),RF (m/s),SR (ug/m3),BP (mmHg),Toluene (ug/m3),Eth-Benzene (ug/m3),MP-Xylene (ug/m3),O Xylene (ug/m3),Xylene (ug/m3)
0,2023-01-17 16:00:00,2023-01-17 17:00:00,39.28,104.02,3.28,78.90,45.29,16.75,NaN,1.09,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-17 17:00:00,2023-01-17 18:00:00,34.92,104.57,10.18,122.68,73.69,11.12,NaN,1.24,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-17 18:00:00,2023-01-17 19:00:00,38.26,115.92,6.59,100.80,59.38,11.88,54.10,1.18,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-17 19:00:00,2023-01-17 20:00:00,36.75,120.53,3.06,60.37,35.43,20.04,57.22,1.11,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-17 20:00:00,2023-01-17 21:00:00,46.86,121.39,4.48,64.05,38.49,19.43,53.66,1.42,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62952 entries, 0 to 62951
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   From Date            62952 non-null  object 
 1   To Date              62952 non-null  object 
 2   PM2.5 (ug/m3)        40370 non-null  float64
 3   PM10 (ug/m3)         38764 non-null  float64
 4   NO (ug/m3)           39554 non-null  float64
 5   NO2 (ug/m3)          39095 non-null  float64
 6   NOx (ppb)            39662 non-null  float64
 7   NH3 (ug/m3)          26710 non-null  float64
 8   SO2 (ug/m3)          39870 non-null  float64
 9   CO (mg/m3)           39793 non-null  float64
 10  Ozone (ug/m3)        23791 non-null  float64
 11  Benzene (ug/m3)      22918 non-null  float64
 12  RH (%)               36195 non-null  float64
 13  WS (m/s)             35664 non-null  float64
 14  WD (degree)          36079 non-null  float64
 15  SR (W/mt2)           34491 non-null 

In [ ]:
# Select features
features = df[['PM2.5 (ug/m3)', 'NO2 (ug/m3)', 'NH3 (ug/m3)', 'CO (mg/m3)', 'AT (degree C)', 'RH (%)']]

# Display the first few rows of the selected data
features.head()

,PM2.5 (ug/m3),NO2 (ug/m3),NH3 (ug/m3),CO (mg/m3),AT (degree C),RH (%)
0,39.28,78.90,16.75,1.09,32.86,51.31
1,34.92,122.68,11.12,1.24,29.95,61.72
2,38.26,100.80,11.88,1.18,27.32,71.30
3,36.75,60.37,20.04,1.11,25.94,80.19
4,46.86,64.05,19.43,1.42,25.38,81.64


In [ ]:
#cleand the unnecesary features
features = ['PM2.5 (ug/m3)', 'NO2 (ug/m3)', 'NH3 (ug/m3)', 'CO (mg/m3)', 'AT (degree C)', 'RH (%)']

df_cleaned = df[features]

print(df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62952 entries, 0 to 62951
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PM2.5 (ug/m3)  40370 non-null  float64
 1   NO2 (ug/m3)    39095 non-null  float64
 2   NH3 (ug/m3)    26710 non-null  float64
 3   CO (mg/m3)     39793 non-null  float64
 4   AT (degree C)  36319 non-null  float64
 5   RH (%)         36195 non-null  float64
dtypes: float64(6)
memory usage: 2.9 MB
None


In [ ]:
df = df_cleaned.interpolate(method='pad')
df = df_cleaned.fillna(df.mean())

Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
normalized_df = pd.DataFrame(scaled_data, columns=df.columns)

In [ ]:
normalized_df.head()

,PM2.5 (ug/m3),NO2 (ug/m3),NH3 (ug/m3),CO (mg/m3),AT (degree C),RH (%)
0,0.056504,0.279097,0.080385,0.113542,0.686145,0.513216
1,0.050230,0.434021,0.053023,0.129167,0.609201,0.617441
2,0.055036,0.356594,0.056717,0.122917,0.539662,0.713356
3,0.052863,0.213525,0.096374,0.115625,0.503173,0.802363
4,0.067410,0.226547,0.093410,0.147917,0.488366,0.816880


In [ ]:
# Rename column in DataFrame 'features'
df = df.rename(columns={'PM2.5 (ug/m3)': 'PM2.5',
                        'NO2 (ug/m3)': 'NO2',
                        'NH3 (ug/m3)': 'NH3',
                        'CO (mg/m3)': 'CO',
                        'AT (degree C)': 'Temperature',
                        'RH (%)': 'Humidity'})

# Define a function to categorize AQI
def categorize_aqi(row):
    if row['PM2.5'] <= 30 and row['NO2'] <= 40 and row['NH3'] <= 200 and row['CO'] <= 1.0:
        return 'Good'
    elif row['PM2.5'] <= 60 and row['NO2'] <= 80 and row['NH3'] <= 400 and row['CO'] <= 2.0:
        return 'Satisfactory'
    elif row['PM2.5'] <= 90 and row['NO2'] <= 180 and row['NH3'] <= 800 and row['CO'] <= 10.0:
        return 'Moderate'
    elif row['PM2.5'] <= 120 and row['NO2'] <= 280 and row['NH3'] <= 1200 and row['CO'] <= 17.0:
        return 'Poor'
    elif row['PM2.5'] <= 250 and row['NO2'] <= 400 and row['NH3'] <= 1800 and row['CO'] <= 34.0:
        return 'Very Poor'
    else:
        return 'Severe'

df['AQI_Category'] = df.apply(categorize_aqi, axis=1)

In [ ]:
# Split data into training and testing sets
X = df[['PM2.5', 'NO2', 'NH3', 'CO', 'Temperature', 'Humidity']]
y = df['AQI_Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Train the model using Random Forest Classifier

In [ ]:
# Train the Random Forest model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

Evaluate the model

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9996028909538559
              precision    recall  f1-score   support

        Good       1.00      1.00      1.00      3109
    Moderate       1.00      1.00      1.00      1194
        Poor       0.98      1.00      0.99        99
Satisfactory       1.00      1.00      1.00      8123
      Severe       1.00      0.78      0.88         9
   Very Poor       0.97      0.98      0.97        57

    accuracy                           1.00     12591
   macro avg       0.99      0.96      0.97     12591
weighted avg       1.00      1.00      1.00     12591



Making predictions on new data

In [ ]:
new_data = pd.DataFrame({
    'PM2.5': [385],
    'NO2': [763],
    'NH3': [2000],
    'CO': [38.5],
    'Temperature': [25],
    'Humidity': [50]
})

new_prediction = model.predict(new_data)
print("Prediksi Kategori AQI:", new_prediction[0])

Prediksi Kategori AQI: Very Poor


Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'AQI_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
#loading the saved model
loaded_model = pickle.load(open('AQI_model.sav', 'rb'))